In [ ]:
# Createa range of years and use pandas html scraping to create data frame of draft classes by year

In [69]:
import pandas as pd
import time 
from bs4 import BeautifulSoup as bs
import requests


In [70]:
draftYears = range(1999,2021)

ColNames=['DraftYear', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To', 'AP1', 'PB', 'St', 
 'CarAV', 'DrAV', 'G', 'PassingCmp', 'PassingAtt', 'PassingYds', 'PassingTD', 
 'PassingInt', 'RushingAtt', 'RushingYds', 'RushingTD', 'ReceivingRec',
 'ReceivingYds', 'ReceivingTD', 'ReceivingSolo', 'Int', 'Sk', 'College/Univ', 
 'CollegeStats'] 

draftDF = pd.DataFrame(columns = ColNames)


for x in draftYears:
    url = f'https://www.pro-football-reference.com/years/{x}/draft.htm'
    dftLst = pd.read_html(url) # returns list
    tempDF = pd.DataFrame(dftLst[0])
    tempDF.insert(0, 'DraftYear', x)
    tempDF.columns = ColNames
    if x == min(draftYears):
        draftDF = tempDF
    else:
        draftDF = draftDF.append(tempDF, ignore_index=True)
    time.sleep(3)
del tempDF
   
#Drop the College Stats column because it is just the text 'College Stats' - use bs to get the actual href and merge in subsequent steps
draftDF.drop(['CollegeStats'], axis=1, inplace = True)   

,DraftYear,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,...,RushingAtt,RushingYds,RushingTD,ReceivingRec,ReceivingYds,ReceivingTD,ReceivingSolo,Int,Sk,College/Univ
0,1999,1,1,CLE,Tim Couch,QB,22,2003,0,0,...,124,556,2,2,-13,0,NaN,NaN,NaN,Kentucky
1,1999,1,2,PHI,Donovan McNabb,QB,22,2011,0,6,...,616,3459,29,5,-7,0,NaN,NaN,NaN,Syracuse
2,1999,1,3,CIN,Akili Smith,QB,24,2002,0,0,...,70,371,1,1,6,0,NaN,NaN,NaN,Oregon
3,1999,1,4,IND,Edgerrin James HOF,RB,21,2009,1,4,...,3028,12246,80,433,3364,11,NaN,NaN,NaN,Miami (FL)
4,1999,1,5,NOR,Ricky Williams,RB,22,2011,1,1,...,2431,10009,66,342,2606,8,1,NaN,NaN,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,2020,7,251,SEA,Stephen Sullivan,TE,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LSU
5731,2020,7,252,DEN,Tyrie Cleveland,WR,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Florida
5732,2020,7,253,MIN,Kyle Hinton,G,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washburn
5733,2020,7,254,DEN,Derrek Tuszka,LB,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North Dakota St.


In [79]:
#use soup to get the href on each row of the table
# note - there are some players that do not have href for college stats so put in the try except
cStatList = []

for x in draftYears:
    url = f'https://www.pro-football-reference.com/years/{x}/draft.htm'
    
    players = bs(requests.get(f'https://www.pro-football-reference.com/years/{x}/draft.htm').text, 'lxml')\
        .findAll('td', attrs={'data-stat':'player'})
    
    hrefs = bs(requests.get(f'https://www.pro-football-reference.com/years/{x}/draft.htm').text, 'lxml')\
        .findAll('td', attrs={'data-stat':'college_link'})
    
    i = 0
    while True:
        tdict ={}
        try:
            tdict['DraftYear'] = x
            tdict['Player'] = players[i].text
            tdict['CollegeStats'] = hrefs[i].a['href']

        except:
            tdict['DraftYear'] = x
            tdict['Player'] = players[i].text
            tdict['CollegeStats'] = ''

        cStatList.append(tdict)

        i+=1
        if(i > len(players)-1):
            break

In [80]:
#Create a datafrom fome the list that was created
statLinkDF = pd.DataFrame(cStatList)
#StatLinkDF.head()

In [81]:
#Merge the data with the draftDF dataframe (use an inner join- will take care of a flaw in the first df)
draftScrapeDF = pd.merge(left=draftDF, right=statLinkDF, left_on=['DraftYear','Player'], right_on=['DraftYear','Player'])

In [83]:
#export the file
draftScrapeDF.to_csv('NFLDraftHistory.csv', index=False, encoding='utf-8')

In [84]:
draftScrapeDF.tail(10)

,DraftYear,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,...,RushingYds,RushingTD,ReceivingRec,ReceivingYds,ReceivingTD,ReceivingSolo,Int,Sk,College/Univ,CollegeStats
5601,2020,7,246,MIA,Malcolm Perry,RB,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Navy,http://www.sports-reference.com/cfb/players/ma...
5602,2020,7,247,NYG,Chris Williamson,CB,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minnesota,http://www.sports-reference.com/cfb/players/ch...
5603,2020,7,248,LAR,Sam Sloman,K,22,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miami (OH),http://www.sports-reference.com/cfb/players/sa...
5604,2020,7,249,MIN,Brian Cole II,S,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi St.,http://www.sports-reference.com/cfb/players/br...
5605,2020,7,250,LAR,Tremayne Anchrum,G,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clemson,http://www.sports-reference.com/cfb/players/tr...
5606,2020,7,251,SEA,Stephen Sullivan,TE,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LSU,http://www.sports-reference.com/cfb/players/st...
5607,2020,7,252,DEN,Tyrie Cleveland,WR,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Florida,http://www.sports-reference.com/cfb/players/ty...
5608,2020,7,253,MIN,Kyle Hinton,G,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washburn,
5609,2020,7,254,DEN,Derrek Tuszka,LB,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North Dakota St.,
5610,2020,7,255,NYG,Tae Crowder,LB,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georgia,http://www.sports-reference.com/cfb/players/ta...
